# Lesson 5: ZK Commitments of Trace Data

In this lesson, we'll examine how cryptographic commitments are created for trace polynomials. This is a key element in ensuring the zero-knowledge property of STARK.

In [ ]:
import numpy as np
import hashlib
from dataclasses import dataclass

@dataclass
class MerkleNode:
    """A node in the Merkle tree."""
    hash: bytes
    left: 'MerkleNode' = None
    right: 'MerkleNode' = None

def hash_value(value):
    """Hashes a value using SHA-256."""
    return hashlib.sha256(str(value).encode()).digest()

def create_merkle_tree(values):
    """Creates a Merkle tree from a list of values.
    
    Args:
        values: List of values to hash
        
    Returns:
        MerkleNode: Root of the tree
    """
    if len(values) == 0:
        return None
    
    # Create leaf nodes
    leaves = [MerkleNode(hash_value(v)) for v in values]
    
    # Build tree bottom-up
    current_level = leaves
    while len(current_level) > 1:
        next_level = []
        for i in range(0, len(current_level), 2):
            left = current_level[i]
            right = current_level[i + 1] if i + 1 < len(current_level) else left
            combined_hash = hashlib.sha256(left.hash + right.hash).digest()
            parent = MerkleNode(combined_hash, left, right)
            next_level.append(parent)
        current_level = next_level
    
    return current_level[0]

## Creating Commitments for our Trace

In [ ]:
# Import functions from previous lessons
from lesson_1_execution_trace import create_fibonacci_trace
from lesson_3_padding_trace import pad_trace_to_power_of_two
from lesson_4_trace_polynomials import create_trace_polynomial

# Create trace and polynomials
trace = create_fibonacci_trace(3, 5)
padded_trace = pad_trace_to_power_of_two(trace)

# Create commitments for each column
commitments = {}
for column in padded_trace.columns[:3]:  # Only for registers
    values = padded_trace[column].values
    merkle_root = create_merkle_tree(values)
    commitments[column] = merkle_root.hash.hex()[:16] + '...'

print("Commitments for trace columns:")
for column, commitment in commitments.items():
    print(f"{column}: {commitment}")

## Creating and Verifying Membership Proofs

In [ ]:
def create_merkle_proof(tree, index, values):
    """Creates a membership proof for an element.
    
    Args:
        tree: Merkle tree root
        index: Element index
        values: List of all values
        
    Returns:
        list: Proof path
    """
    proof = []
    current_level_size = len(values)
    current_index = index
    current_node = tree
    
    while current_level_size > 1:
        is_right = current_index % 2 == 1
        sibling = current_node.right if not is_right else current_node.left
        proof.append((sibling.hash.hex()[:16] + '...', 'right' if is_right else 'left'))
        
        current_node = current_node.left if not is_right else current_node.right
        current_index //= 2
        current_level_size = (current_level_size + 1) // 2
        
    return proof

# Create and verify proof for a specific value
column = 'Register 1'
index = 2
value = padded_trace[column].values[index]
merkle_root = create_merkle_tree(padded_trace[column].values)
proof = create_merkle_proof(merkle_root, index, padded_trace[column].values)

print(f"\nProof for {column}[{index}] = {value}:")
for hash_value, direction in proof:
    print(f"- {direction}: {hash_value}")

## Important Aspects of ZK Commitments

1. **Commitment Properties**:
   - **Binding**: Cannot change data after commitment
   - **Hiding**: Commitment doesn't reveal data
   - **Compactness**: Commitment size independent of data size

2. **Use in STARK**:
   - Allows verifier to check parts of trace without access to all data
   - Ensures zero-knowledge property
   - Enables efficient verification of large traces

3. **Efficiency**:
   - Merkle tree creation: O(n)
   - Proof size: O(log n)
   - Verification time: O(log n)

In the next lesson, we'll examine how these commitments are used together with polynomial constraints.